# Homework 7: contraction mappings

## Exercise 1.
This notebook implements an iterative procedure for contraction mapping over a pricing function for an asset pricing problem. We want to solve for the pricing function $p(\cdot)$ in the problem:

\begin{align}
p(x) = \beta  \sum_{y \in S}[d(y) + p(y)] q_i(x,y)
\end{align}

We use the Theorem from Lucas (1978), which seems to be an application of the contraction mapping theorem. 

Before we begin, first note that we can stack the dividend as a function of the state space, $d(x)$, as a column vector: $d = [1, 0, 0]'$. This way, our price function can also be described by a vector $p = [p_1, p_2, p_3]'$ where $p_i$ describes the price of the asset in state $i$.

Using Python, first we import some useful functions.  

In [1]:
import numpy as np
from numpy.linalg import norm

Now, set up the parameters of the model: the discount factor $\beta$, the vector of dividends $d$ given the state, and the transition matrices $q_1, q_2$ (built from the probabilities $q_1(x,y), q_2(x,y)$. We also set the maximum number of 

In [2]:
beta = 0.98
d = np.array([1,0,0])
d.shape = 3,1
q1 = np.array([[0.971, 0.029, 0.00],[0.145, 0.778, 0.077],[0.00, 0.508, 0.492]])
q2 = np.array([[0.871, 0.029, 0.1],[0.145, 0.778, 0.077],[0.00, 0.508, 0.492]])

p0 = np.array([1,1,1])
p0.shape = 3,1
p0

maxiter = 10000
tol = 1e-5

Now create the function that for any given price function (/vector) returns the contraction applied to the function: $Tp$. Note that here I am pre-assigning the parameter $\beta$, the dividend vector $d$, and the transition probability matrix $q$ to the function `contract`. This way I need only use one input -- the current guess for the pricing vector -- each time I call the function `contract`.

In [3]:
def create_contract(beta, d, q):
    def contract(p):
        out = beta*q@(d + p)
        return out
    return contract

contract1 = create_contract(beta,d,q1)
contract2 = create_contract(beta,d,q2)

Create a function `iterate` that takes as inputs: an initial guess $p0$, a maximum number of iterations $K$, a norm tolerance $Ttol$, and a contraction function (parameterized by $\beta$, $d$, and $q$, as above). `iterate` iterates on the contraction mapping, and returns the pricing function solution.

In [4]:
def create_iterate(p0,maxiter,tol):
    def iterate(contract):
        Told = contract(p0)
        for k in range(maxiter):
            Tnew = contract(Told)
            if norm(Tnew - Told, np.inf) < tol:
                print('Converged in %i iterations\n' % k)
                break    
            Told = Tnew
        return Tnew
    return iterate
    
iterate = create_iterate(p0,maxiter,tol)

Now perform the contraction iteration procedure for the first transition probability matrix, $q_1$:

In [5]:
p1 = iterate(contract1)
p1

Converged in 557 iterations



array([[ 40.75929094],
       [ 35.96039902],
       [ 34.57151895]])

Now perform the contraction iteration procedure for the second transition probability matrix, $q_2$:

In [6]:
p2 = iterate(contract2)
p2

Converged in 527 iterations



array([[ 23.50065386],
       [ 21.09835023],
       [ 20.28347185]])

Notice that the pricing function solution returns larger prices in every state of the world under the probability transition matrix $q_1$ than under $q_2$. This is because in the second case, there is a positive probability of transitioning from the Normal Growth (NG) state to the Severe Recession (SR) state. Once in the SR state, there is a zero dividend, but also zero probability of transitioning back to the NG state with a dividend of one. Once in this SR state, you must first transition through the Medium Recession state (with probability 0.508), and then transition into the NG state (with probability 0.145) to get back to earning non-zero dividends.

Thus, the probability of entering a state of persistently zero dividends (from any state) is higher under $q_2$ than under $q_1$. For this reason, the price of the asset is lower in all states.

Note, too, that this occurs under risk-neutral preferences. Thus the change in price reflects the change in expected values only. With risk-averse preferences, instead, the fall in asset prices would be much larger. 

## Exercise 2.

Now we want to solve the heterogeneous beliefs problem, where the pricing function is given by

\begin{align}
p(x) = \beta \underset{i \in \{1,2\}}{\max} \left\{ \sum_{y \in S}[d(y) + p(y)] q_i(x,y)   \right\}
\end{align}

We first have to show that the equation has a unique solution in $b \mathbb{R}^S$, which we can do simply by showing that the linear operator $Tp$ satisfies the assumptions of Theorem 0.1. Namely, that $T: b \mathbb{R}^S \rightarrow b \mathbb{R}^S $ and that $T$ is a contraction. 

First, we show that the $T$ operator is bounded:
\begin{align}
\lvert Tp(x) \rvert & = \beta \left\lvert \underset{i \in \{1,2\}}{\max} \sum_{y \in S}[d(y) + p(y)] q_i(x,y)   \right\rvert \\
& \leq \beta \underset{i \in \{1,2\}}{\max} \left\lvert \sum_{y \in S}[d(y) + p(y)] q_i(x,y)  \right\rvert \\
& \leq \beta \underset{i \in \{1,2\}}{\max} \sum_{y \in S} \left\lvert d(y) + p(y) \right\rvert q_i(x,y) \\
& \leq \beta \underset{i \in \{1,2\}}{\max} \sum_{y \in S} ( \| d \|_{\infty} + \| p \|_{\infty} ) q_i(x,y) \\
& = \beta (\| d \|_{\infty} + \| p \|_{\infty} ) 
\end{align}
where the final equality follows from the fact that for both agents, probabilities must sum to one. Notice that this is a contstant bound for all $x$ (a uniform bound), and so $\|Tp\|_{\infty} < \infty $ and so $T: b \mathbb{R}^S \rightarrow b \mathbb{R}^S $.

Second, we show that $T$ is a contraction:
\begin{align}
\lvert Tp(x) - Tp'(x) \rvert & = \beta \left\lvert \underset{i \in \{1,2\}}{\max} \sum_{y \in S}[d(y) + p(y)] q_i(x,y) - \underset{i \in \{1,2\}}{\max} \sum_{y \in S}[d(y) + p'(y)] q_i(x,y)   \right\rvert \\
& \leq \beta \underset{i \in \{1,2\}}{\max} \left\lvert  \sum_{y \in S}[d(y) + p(y)] q_i(x,y) - \sum_{y \in S}[d(y) + p'(y)] q_i(x,y)   \right\rvert \\
& = \beta \underset{i \in \{1,2\}}{\max} \left\lvert  \sum_{y \in S}[p(y) - p'(y)] q_i(x,y)   \right\rvert \\
& \leq \beta \underset{i \in \{1,2\}}{\max}  \sum_{y \in S} \left\lvert p(y) - p'(y) \right\rvert q_i(x,y) \\
& \leq \beta \underset{i \in \{1,2\}}{\max}  \sum_{y \in S} \| p - p' \|_{\infty}  q_i(x,y) \\
& = \beta \| p - p' \|_{\infty}
\end{align}
Again where the final equality follows from the fact that for both agents, probabilities must sum to one. And again, since the bound is for any $x$, we have that 
\begin{align}
\| Tp - Tp' \|_{\infty} \leq \beta \| p - p' \|_{\infty}
\end{align}
And so $T$ is a contraction on $(b \mathbb{R}^S,d_{\infty})$. Thus, the theorem applies, and our pricing function under heterogeneous beliefs has a unique solution. 

### Exercise 3.

Now we want to solve the problem when the pricing function reflects the fact that the marginal investor in a given state $x$ is the one with the highest probability weighted valuation of the asset. The pricing function is given by

\begin{align}
p(x) = \beta \underset{i \in \{1,2\}}{\max} \left\{ \sum_{y \in S}[d(y) + p(y)] q_i(x,y)   \right\}
\end{align}

We simply have to adjust the above code to take account of the maximum.

In [7]:
def iterate_het(p0,maxiter,tol,contract1,contract2):
    Told = np.maximum(contract1(p0),contract2(p0))
    for k in range(maxiter):
        Tnew = np.maximum(contract1(Told),contract2(Told))
        if norm(Tnew - Told, np.inf) < tol:
            print('Converged in %i iterations\n' % k)
            break    
        Told = Tnew
    return Tnew

In [8]:
phet = iterate_het(p0,maxiter,tol,contract1,contract2)
phet

Converged in 557 iterations



array([[ 40.75929094],
       [ 35.96039902],
       [ 34.57151895]])

The agent with the larger probability belief on remaining in state NG (agent 1) puts more value on the asset in every state, and so is the marginal agent with respect to pricing the asset. As such, the pricing function under heterogeneous agents is the same as the pricing function under agent 1's beliefs alone. 

## Extra credit question

We are asked to prove the hint we used in the proof in exercise 2. Namely, show that:
\begin{align}
\left\lvert \underset{i}{\max} a_i - \underset{i}{\max} b_i \right\rvert \leq \underset{i}{\max} \left\lvert a_i - b_i \right\rvert
\end{align}

A proof proceeds as follows: 
\begin{align}
\underset{i}{\max} \left\lvert a_i - b_i \right\rvert & \geq  \left\lvert \underset{i}{\max} \left( a_i - b_i \right) \right\rvert \\
& \geq \left\lvert \underset{i}{\max} \left( a_i - \left( \underset{j}{\max} b_j \right) \right) \right\rvert \\
& \geq \left\lvert \underset{i}{\max} a_i - \underset{i}{\max} b_i \right\rvert
\end{align}

Where the final inequality follows from the fact that $ ( \underset{j}{\max} b_j ) $ is a constant relative to the outer maximum's index, and so can be taken outside of the outer maximum. I think the same argument follows with an arbitrary index set $i$, and so the same proof should follow but replacing max with sup. 
